This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([150.37102595, 202.0924573 , 325.49788774, ...,  19.41019935,
        47.20790294,   0.        ]), array([ 75.18551297, 101.04618047, 162.74894387, ...,   9.70509967,
        23.60395147,   0.        ]), OrderedDict([('R2', 0.9983371190298685), ('root_mean_squared_error', 71.97350273959277), ('mean_absolute_error', 53.066494087867795), ('rmse_over_stdev', 0.7404723672954406)]), None)
test_quad = (array([1.77283020e+02, 2.39214774e+02, 2.14185752e+02, 1.46594936e+02,
       1.40876460e+02, 9.96717956e+01, 9.45736257e+01, 9.62792101e+01,
       2.02290968e+02, 7.75902143e+01, 1.02665208e+02, 1.72420203e+02,
       1.22487016e+02, 1.78669886e+02, 1.87709915e+02, 1.79995444e+02,
       1.53899276e+02, 1.25776102e+02, 3.69780134e+02, 3.02985863e+02,
       3.19706165e+02, 2.29722452e+02, 2.27914069e+02, 1.72040539e+02,
       1.19298374e+02, 1.24022947e+02, 7.70450205e+01, 7.45855168e+01,
       1.74631224e+02, 1.31350152e+02, 4.50731986e+01, 1.09473901e+02,
       0.00000000e+00, 2.56639133e+01, 3.90426011e+02, 2.36177315e+02,
       8.39207969e+01, 1.56229730e+02, 7.39283454e+01, 6.86792014e+01,
       1.13135302e+02, 7.34978297e+01, 1.63297865e+02, 3.20094037e+01,
       0.00000000e+00, 6.94361418e+01, 9.24561079e+01, 1.56627930e+02,
       6.36890317e+01, 5.90096117e+00, 1.56215250e+01, 7.38023139e+01,
       3.57160729e+02, 2.91632073e+01, 1.49611167e+02, 3.07830539e+02,
       2.26412639e+01, 8.03088872e+01, 5.18644462e+01, 1.39599684e+02,
       1.51797441e+02, 2.89712978e+02, 1.18302658e+02, 2.94893175e+02,
       3.39191772e+01, 3.93315709e+01, 2.99490808e+01, 2.58389489e+01,
       0.00000000e+00, 0.00000000e+00, 4.51430137e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.70304392e+01, 0.00000000e+00,
       0.00000000e+00, 1.39370004e+02, 3.20782685e+01, 3.35683107e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.49408401e+01,
       7.74558848e+00, 6.05219124e+01, 3.21439514e+01, 1.61615856e+02,
       1.31304680e+02, 1.08979858e+02, 8.64067635e+01, 0.00000000e+00,
       0.00000000e+00, 1.28935952e+02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.44184734e+01, 6.21108106e+01, 8.83802997e+01, 2.41335092e+01,
       6.53557069e+01, 4.48563810e+01, 0.00000000e+00, 1.33902387e+01,
       6.12137026e+01, 4.85711869e+01, 4.91573257e+01, 8.68896540e+01,
       3.61795100e+01, 1.04542615e+02, 1.05693165e+02, 8.65356568e+01,
       8.04591080e+01, 8.51942082e+01, 1.00949225e+02, 1.26886312e+02,
       1.85404211e+02, 0.00000000e+00, 2.02289181e-01, 1.38892855e+02,
       1.37126059e+02, 1.26881899e+02, 1.70388617e+02, 8.98167378e+01,
       9.36717096e+01, 1.50078284e+01, 1.93409720e+01, 0.00000000e+00,
       1.24480773e+01, 4.70862505e+01, 1.27378189e+02, 1.14554644e+02,
       1.66187429e+02, 1.06702335e+02, 2.76340269e+02, 0.00000000e+00,
       4.84645306e+01, 2.94415703e+02, 1.65778433e+02, 1.39763133e+02,
       3.15202492e+01, 1.40538550e+02, 2.29064683e+02, 1.13901487e+02,
       4.80411705e+01, 2.53031123e+01, 3.81465155e+01, 6.49749186e+01,
       4.22355880e+01, 2.62303388e+01, 9.53324888e+01, 9.10665100e+00,
       0.00000000e+00, 1.05110827e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.84401941e+01, 5.54728596e+01, 5.13197137e+01,
       5.25266742e+01, 4.81837797e+01, 3.08078053e+01, 2.02393592e+02,
       5.65407126e+01, 3.81230146e+01, 5.46661575e+01, 5.87368740e+01,
       1.29976759e+02, 4.16645268e+01, 2.82496436e+01, 3.40111139e+01,
       0.00000000e+00, 6.50315409e+01, 1.89559087e+01, 3.92079232e+01,
       1.52429429e+02, 9.46538849e+01, 8.30269047e+01, 0.00000000e+00,
       4.00673869e+01, 2.50852147e+01, 3.11645320e+01, 2.75959946e+01,
       0.00000000e+00, 9.54966739e+01, 4.94252903e+01, 1.05442368e+02,
       0.00000000e+00, 0.00000000e+00, 6.89012756e+00, 0.00000000e+00,
       0.00000000e+00, 7.05592531e+00, 5.40607889e+01, 6.90074178e+01,
       0.00000000e+00, 0.00000000e+00, 1.29601532e+02, 1.55663046e+02,
       1.36431444e+02, 1.89093084e+02, 1.79654132e+02, 2.07904807e+02,
       2.34660651e+02, 3.36224697e+02, 9.15340044e+01, 1.16180977e+02,
       1.78939785e+02, 1.14272223e+02, 1.13869162e+02, 2.43428371e+02,
       1.14407716e+02, 1.86026530e+02, 1.30347188e+02, 1.26700822e+02,
       2.44148911e+02, 1.55694637e+02, 2.59886084e+02, 2.20161511e+02,
       1.96409134e+02, 1.44708713e+02, 1.91836674e+02, 1.41535230e+02,
       2.10335728e+02, 1.03165153e+02, 1.35390845e+02, 2.51196620e+02,
       2.37062344e+02, 2.73613179e+02, 3.62461054e+02, 1.77346156e+02,
       2.26507616e+02, 3.23822637e+02, 2.04046571e+02, 2.23816699e+02,
       3.40437655e+02, 2.55478983e+02, 3.18399908e+02, 5.98511381e+01,
       1.43517205e+02, 8.23163755e+01, 1.97124130e+02, 9.69915119e+01,
       2.10107835e+02, 1.61293351e+02, 9.69267962e+01, 9.83944608e+01,
       1.93969856e+02, 1.35408933e+02, 1.31977595e+02, 1.83449190e+02,
       1.02794584e+02, 1.60233988e+02, 3.64707101e+02, 1.86717789e+02,
       1.44889042e+02, 3.17205543e+02, 3.17356871e+02, 3.28541723e+02,
       3.23634151e+02, 3.46087908e+02, 1.20403530e+02, 1.41453455e+02,
       7.10448978e+01, 8.95779086e+01, 1.24842572e+02, 8.85969515e+01,
       1.38575932e+02, 1.36443088e+02, 1.47525298e+02, 6.40241718e+01,
       5.55045542e+01, 1.38091226e+02, 7.74665625e+01, 1.33883367e+02,
       1.32392433e+02, 1.26818170e+02, 5.68526124e+01, 1.84907725e+02,
       1.25571144e+02, 2.72871580e+02, 2.10329787e+02, 2.83042429e+02,
       2.27717761e+02, 2.47909595e+02, 2.42965465e+02, 2.00760493e+02,
       2.04542101e+02, 3.18785354e+02, 2.71906598e+02, 2.09325439e+02,
       1.35426489e+02, 8.10050964e+01, 5.74165434e+01, 1.04682468e+02,
       9.45208382e+01, 9.43913417e+01, 1.78159873e+02, 9.64735725e+01,
       2.72145096e+01, 6.07829098e+01, 2.42317220e+01, 4.96708469e+01,
       7.34595746e+01, 7.19683460e+01, 1.82785945e+01, 4.28892788e+01,
       3.27734114e+01, 1.35840083e+02, 1.02112315e+02, 1.73385595e+02,
       1.84215977e+02, 1.97176332e+02, 1.84779292e+02, 1.70292754e+02,
       1.63409286e+02, 1.63793911e+02, 1.11384136e+02, 1.54672899e+02,
       1.15407866e+02, 1.81387575e+02, 0.00000000e+00, 2.98870605e+01,
       7.73860277e+01, 9.14620807e+01, 8.35686242e+01, 9.06089707e+01,
       2.71319339e+02, 1.09488956e+02, 9.60360384e+01, 2.58828051e+02,
       1.17249168e+02, 1.37791901e+02, 1.85661694e+02, 1.12287386e+02,
       1.40364781e+02, 1.37369485e+02, 1.21692419e+02, 3.35321293e+02,
       9.37481720e+01, 2.63543371e+02, 4.21335070e+01, 3.63667394e+01,
       2.90221571e+02, 2.71330052e+01, 4.45796202e+00, 1.91667256e+01,
       9.53050436e+01, 3.73225941e+01, 1.12949106e+02, 5.04471564e+01,
       1.34244899e+02, 1.43075475e+02, 1.40737465e+02, 1.40488835e+02,
       1.04314376e+01, 1.43739984e+02, 1.61966268e+02, 1.78944559e+02,
       1.82449930e+02, 3.87496682e+01, 3.50703870e+01, 6.20947510e+01,
       6.92239663e+00, 0.00000000e+00, 0.00000000e+00, 6.23247673e+01,
       4.92192870e+01, 5.92192098e+01, 7.32347672e+01, 1.60334633e+02,
       5.19157130e+01, 2.84812922e+01, 2.83147240e+01, 2.55044144e+00,
       3.23684503e+01, 4.56348093e+01, 0.00000000e+00, 3.86517633e+01,
       2.99157066e+01, 7.07659236e+01, 2.67038101e+01, 3.37558922e+01,
       3.74680233e+01, 8.17268550e+01, 7.65032664e+01, 7.90798034e+01,
       7.72377141e+01, 6.23324679e+01, 5.22740325e+01, 1.21693503e+02,
       3.81925114e+01, 5.21415070e+01, 0.00000000e+00, 4.44093613e+01,
       4.39454900e+01, 3.91274706e+01, 1.90870878e+02, 1.66347669e+02,
       8.65192126e+01, 8.25826337e+01, 1.77019953e+01, 1.05624926e+02,
       3.62234796e+01, 1.42323884e+02, 3.48683785e+01, 4.63580983e+01,
       3.07458097e+01, 5.78409146e+01, 4.67667976e+01, 3.53717988e+01,
       5.24254612e+01, 4.66982172e+01, 2.28817925e+01, 8.97135102e+01,
       7.80670028e+01, 2.19120080e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.88957798e+01, 0.00000000e+00,
       2.38090660e+01]), array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 4.49083689e+001,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 3.06415343e-199, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 3.71719563e-199, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 8.18502491e+001, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000]), OrderedDict([('R2', 0.0004923358185209281), ('root_mean_squared_error', 137.15342475831355), ('mean_absolute_error', 105.32252843800947), ('rmse_over_stdev', 1.4110515293512051)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge_DoNothing_DoNothing_GridSearch/RepeatedKFold/split_9'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
